In [2]:
# ! pip install coremltools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 5.4 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.24.4
    Uninstalling protobuf-4.24.4:
      Successfully uninstalled protobuf-4.24.4


In [3]:
import math
import json
import numpy as np
import pandas as pd
import coremltools as ct
from datetime import datetime
from collections import defaultdict

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

scikit-learn version 1.2.2 is not supported. Minimum required version: 0.17. Maximum required version: 1.1.2. Disabling scikit-learn conversion API.
TensorFlow version 2.14.0 has not been tested with coremltools. You may run into unexpected errors. TensorFlow 2.12.0 is the most recent version that has been tested.


In [4]:
STAGE = {
    'In Bed': 0,
    'Awake': 1,
    'Asleep': 2,
    'REM': 3,
    'Core': 4,
    'Deep': 5,
    'Unknown': 6
}

In [5]:
def load_json(path: str) -> dict:
    with open(path, 'r') as file:
        return json.load(file)

def parse_time(time: str) -> datetime:
    return datetime.strptime(time, '%Y-%m-%d %H:%M:%S')

def to_min(cur_time: datetime, start_time: datetime) -> int:
    return int((cur_time - start_time).total_seconds()//60)

def process_sleep_data(sleep_data: dict) -> dict:
    stage_data = defaultdict(dict)

    for data in sleep_data:
        start_time = parse_time(data['start_time'])
        end_time = parse_time(data['end_time'])
        date = end_time.strftime('%Y-%m-%d')

        if not date in stage_data: stage_data[date]['stages'] = list()
        stage_data[date]['stages'].append({
            'start_time': start_time,
            'end_time': end_time,
            'stage': STAGE[data['stage']]
        })

    for date, data in stage_data.items():
        stage_data[date]['stages'] = sorted(data['stages'], key=lambda x: x['start_time'])
        stage_data[date]['start_time'] = stage_data[date]['stages'][0]['start_time']
        stage_data[date]['end_time'] = stage_data[date]['stages'][-1]['end_time']
        stage_data[date]['day_of_week'] = stage_data[date]['start_time'].weekday()

    return stage_data

def post_process(stage_data: dict) -> pd.DataFrame:
    ml_data = list()
    for date, data in stage_data.items():
        for item in data['stages']:
            start_time = to_min(item['start_time'], data['start_time'])
            end_time = to_min(item['end_time'], data['start_time'])
            for time in range(start_time, end_time):
                ml_data.append((
                    date,
                    time,
                    item['stage']
                ))
    return pd.DataFrame(ml_data, columns=['date', 'day_of_week', 'time', 'stage'])

In [6]:
data = load_json('./sleepData.json')
data = data['sleep_data']

stage_data = process_sleep_data(data)
ml_data = post_process(stage_data)

ml_data

ValueError: 4 columns passed, passed data had 3 columns

In [6]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
print('Current backend accelerator:', device)

Current backend accelerator: mps


In [7]:
# Split data into features and target
X = ml_data[['day_of_week', 'time']].values
y = ml_data['stage'].values

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a PyTorch Dataset
class SleepStageDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return torch.tensor(self.features[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)

train_dataset = SleepStageDataset(X_train, y_train)
val_dataset = SleepStageDataset(X_val, y_val)

# DataLoader
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [8]:
class TransformerModel(nn.Module):
    def __init__(self, num_features, num_classes, dim_model=128, num_heads=4, num_encoder_layers=3, dropout_rate=0.1):
        super(TransformerModel, self).__init__()
        self.embedding_layer = nn.Linear(num_features, dim_model)

        self.positional_encoding = PositionalEncoding(dim_model, dropout_rate)
        
        encoder_layer = nn.TransformerEncoderLayer(d_model=dim_model, nhead=num_heads, dropout=dropout_rate)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(dim_model, num_classes)

    def forward(self, src):
        src = self.embedding_layer(src)
        src = src.unsqueeze(1)  # Add batch dimension
        src = self.positional_encoding(src)
        output = self.transformer_encoder(src)
        output = output.squeeze(1)  # Remove batch dimension
        output = self.dropout(output)
        output = self.classifier(output)
        return output

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [9]:
def train(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}')

def evaluate(model, val_loader):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for features, labels in val_loader:
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total}%')

In [10]:
# Model parameters
num_features = 2  # day_of_week and time
num_classes = len(set(y_train))  # Assuming y_train is accessible here

# Initialize the model, criterion, and optimizer
model = TransformerModel(num_features, num_classes)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train and evaluate the model
train(model, train_loader, criterion, optimizer, num_epochs=10)
evaluate(model, val_loader)

/Users/nitro/miniconda3/envs/cs178/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1, Loss: 1.0038286174064155
Epoch 2, Loss: 0.9987461775893995
Epoch 3, Loss: 0.9988278262033768
Epoch 4, Loss: 0.9987478831362543
Epoch 5, Loss: 0.998903670153775
Epoch 6, Loss: 0.9988938988590281
Epoch 7, Loss: 0.998983971906837
Epoch 8, Loss: 0.9988751759719566
Epoch 9, Loss: 0.9988846839708689
Epoch 10, Loss: 0.9989667202795549
Accuracy: 70.21849982826495%


In [11]:
# Convert to TorchScript
model.eval()
model.to('cpu')
torch.save(model, 'pytorchmodel.pth')

X_data, _ = list(train_loader)[0]

traced_model = torch.jit.trace(model, X_data)
traced_model.save('traced_model.pt')

model = ct.convert(
    traced_model,
    convert_to='mlprogram',
    inputs=[ct.TensorType(shape=X_data.shape)]
)
 
# Save the converted model.
model.save('sleepCoreML.mlpackage')

Running MIL default pipeline:   0%|                           | 0/71 [00:00<?, ? passes/s]/Users/nitro/miniconda3/envs/cs178/lib/python3.10/site-packages/coremltools/converters/mil/mil/passes/defs/preprocess.py:239: UserWarning: Input, 'src.1', of the source model, has been renamed to 'src_1' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Running MIL backend_mlprogram pipeline: 100%|██████| 12/12 [00:00<00:00, 1873.01 passes/s]
